# Megadetector V4

Dieses Notebook enthält einen Object Detector für Objekte der Klasse Mensch (human) und Tier (animal).

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys

Für die Einbindung des Object Detectors wird die ältere TensorFlow-Version 1.13.1 benötigt.
Dafür wird die aktuelle Version deinstalliert.

Die Funktionen und Klassen aus dem Package *humanfriendly* können verwendet werden, um Textschnittstellen benutzerfreundlicher zu gestalten.

Die Pythonlibrary *jsonpickle* dient zur Serialisierung und Deserialisierung von komplexen Python-Objekten in und aus JSON.

In [ ]:
!pip uninstall tensorflow-y


In [ ]:
!pip install tensorflow==1.13.1 humanfriendly jsonpickle

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
import tensorflow as tf
print(tf.__version__)


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1


In [ ]:
tf.test.gpu_device_name()

''

Die neuste Version des MegaDetector Modells wird über den Link [md_v4.1.0.pb](https://lilablobssc.blob.core.windows.net/models/camera_traps/megadetector/md_v4.1.0/md_v4.1.0.pb) gedownloaded.

In [ ]:

!wget -O megadetector_v4_1_0.pb "https://lilablobssc.blob.core.windows.net/models/camera_traps/megadetector/md_v4.1.0/md_v4.1.0.pb"

--2021-02-23 14:25:29--  https://lilablobssc.blob.core.windows.net/models/camera_traps/megadetector/md_v4.1.0/md_v4.1.0.pb
Resolving lilablobssc.blob.core.windows.net (lilablobssc.blob.core.windows.net)... 52.239.159.84
Connecting to lilablobssc.blob.core.windows.net (lilablobssc.blob.core.windows.net)|52.239.159.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 245590501 (234M) [application/octet-stream]
Saving to: ‘megadetector_v4_1_0.pb’

megadetector_v4_1_0 100%[===================>] 234.21M   932KB/s    in 3m 45s  

2021-02-23 14:29:15 (1.04 MB/s) - ‘megadetector_v4_1_0.pb’ saved [245590501/245590501]



Für ein einfacheres Datenhandling und die Ausführung des Modells werden die geforderten GitHub Repositories geclont.

Benötigt werden die neuste Version der Microsoft AI for Earth "utilities" und "Camera Traps".

In [ ]:
!git clone https://github.com/microsoft/CameraTraps/
!git clone https://github.com/microsoft/ai4eutils/

fatal: destination path 'CameraTraps' already exists and is not an empty directory.
fatal: destination path 'ai4eutils' already exists and is not an empty directory.


In [ ]:
print(os.listdir(os.getcwd()))

['run_tf_detector_batch.py', 'megadetector_v4_1_0.pb', 'ai4eutils', 'output.json', '__notebook_source__.ipynb', 'visualize_detector_output.py', 'CameraTraps', 'images']


Außerdem werden die Pythonskripte für die Ausführung des Modells und die Visualisierung der Ergebnisse in das Working Directory kopiert.

In [ ]:
!cp /kaggle/working/CameraTraps/detection/run_tf_detector_batch.py .
!cp /kaggle/working/CameraTraps/visualization/visualize_detector_output.py .

Um später die Module aus jedem Arbeitsverzeichnis importieren zu können, werden die geklonten GitHub-Ordner zur Umgebungsvariable PYTHONPATH hinzugefügt. 

In [ ]:
os.environ['PYTHONPATH'] += ":/kaggle/working/ai4eutils"
os.environ['PYTHONPATH'] += ":/kaggle/working/CameraTraps"

!echo "PYTHONPATH: $PYTHONPATH"

PYTHONPATH: /kaggle/lib/kagglegym:/kaggle/lib:/kaggle/input/iwildcam-2020-fgvc7:/kaggle/working/ai4eutils:/kaggle/working/CameraTraps


# Object Detection

Da der Detector etwa 1 Sekunde pro Bild benötigt, das heißt, etwa 3600 Bilder pro Stunde verarbeiten kann, wird er  im Folgenden beispielhaft auf 20 Testbilder angewandt.

Um das Pythonskript [run_tf_detector_batch.py](https://github.com/microsoft/CameraTraps/blob/master/detection/run_tf_detector_batch.py) aus dem Camera Traps Repository auszuführen, sind folgende Argumente nötig:
1. Einen Ordner mit den Bildern, auf denen die Object Detection ausgeführt werden soll
2. Eine Outputdatei in JSON Format, welche die ermittelten Bilddaten enthält.


In [ ]:
images_dir = '../input/iwildcam-2020-fgvc7/test'

test_images = os.listdir(images_dir)
#test batch
test_images_batch = test_images[:20] 

for image in test_images_batch:
    !cp ../input/iwildcam-2020-fgvc7/test/{image} /kaggle/working/images
    
# choose a location for the output JSON file
output_file_path = '/output/kaggle/working/output.json'

In [ ]:
os.chdir("/kaggle/working")

In [ ]:
images_directory = '/kaggle/working/images/'
# choose a location for the output JSON file
output_file_path = '/kaggle/working/output.json'

Im nächsten Schritt wird die Object Detection ausgeführt. Dabei wird der Wert der Variable output_file_path übergeben, um sich später zur Visualisierung auf den Pfad des Outputfile beziehen zu können.

Das Modell basiert auf einem Faster-RCNN mit einem Inception ResNet v2 Basisnetzwerk. Das ist eine konvolutionale neuronale Architektur, die auf der Inception-Familie von Architekturen aufbaut, aber residuale Verbindungen einbezieht und damit die Filterverkettungsstufe der Inception-Architektur ersetzt.
Das Modell wurde mit der TensorFlow Object Detection API trainiert. 

In [ ]:
#run megadetector
!python run_tf_detector_batch.py megadetector_v4_1_0.pb "$images_directory" "$output_file_path" --recursive

TensorFlow version: 1.13.1
2021-02-23 14:35:25.256370: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-02-23 14:35:25.275185: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-02-23 14:35:25.275771: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x557a1a1bd730 executing computations on platform Host. Devices:
2021-02-23 14:35:25.275823: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
Is GPU available? tf.test.is_gpu_available: False
TensorFlow version: 1.13.1
tf.test.is_gpu_available: False
40 image files found in the input directory
TFDetector: Loading graph...
TFDetector: Detection graph loaded.
Loaded model in 9.46 seconds
  0%|                                                    | 0/40 [00:00<?, ?it/s]Processing image /kaggle/working/images/anno_~kaggle~working~images~8c

100%|███████████████████████████████████████████| 40/40 [19:35<00:00, 29.38s/it]
Finished inference in 19 minutes and 44.71 seconds
Output file saved at /kaggle/working/output.json
Done!


# Visualization

Mit [visualize_detector_output.py](https://github.com/microsoft/CameraTraps/blob/master/visualization/visualize_detector_output.py) aus dem Ordner visualization des Camera Traps Repository lassen sich die Ausgaben des Detectors auf unseren eingelesenen Bildern anzeigen.

In [ ]:
visualization_dir = '/kaggle/working/images/'

In [ ]:
!python visualize_detector_output.py "$output_file_path" "$visualization_dir" --confidence 0.8 --images_dir "$images_dir"

detection_categories provided
Detector output file contains 40 entries.
Starting to annotate the images...
100%|███████████████████████████████████████████| 40/40 [00:02<00:00, 18.23it/s]
Rendered detection results on 40 images, saved to /kaggle/working/images/.


In [ ]:
import os
from PIL import Image

Hier werden anschließend jeweils die Bilder als Original, Ausschnitt und mit Detection (anno) ausgegeben.



In [ ]:
for viz_file_name in os.listdir(visualization_dir):
  #print(viz_file_name)
  im = Image.open(os.path.join(visualization_dir, viz_file_name))
  #display(im)  

![grafik.png](attachment:grafik.png)

Aufgrund der beschränkten Kernel Source Größe findet man neben dem hier angezeigten Beispielbild auf GitHub im Ordner [detector](https://github.com/pds2021/capstone-group_2/tree/main/detector) weitere Beispielbilder zum Object Detector.

# Output

Die Outputdatei [output.json](https://github.com/pds2021/capstone-group_2/blob/main/detector/output.json) enhält für jedes Bild den Dateinamen und die maximale Detection-Konfidenz, sowie die Boundingbox und Konfidenz pro erkanntes Tier. 

In [ ]:
import json

In [ ]:
with open(r'./output.json') as json_file:
    data = json.load(json_file)

In [ ]:
data.keys()

dict_keys(['images', 'detection_categories', 'info'])

In [ ]:
data['images']

[{'file': '/kaggle/working/images/anno_~kaggle~working~images~8c712ba8-21bc-11ea-a13a-137349068a90.jpg',
  'max_detection_conf': 0.11,
  'detections': [{'category': '1',
    'conf': 0.11,
    'bbox': [0.3448, 0.7357, 0.03945, 0.06361]}]},
 {'file': '/kaggle/working/images/905cdb0e-21bc-11ea-a13a-137349068a90.jpg',
  'max_detection_conf': 0.999,
  'detections': [{'category': '1',
    'conf': 0.999,
    'bbox': [0.04335, 0.123, 0.1879, 0.7156]}]},
 {'file': '/kaggle/working/images/9310face-21bc-11ea-a13a-137349068a90.jpg',
  'max_detection_conf': 0.999,
  'detections': [{'category': '1',
    'conf': 0.999,
    'bbox': [0.7618, 0.251, 0.2362, 0.3822]}]},
 {'file': '/kaggle/working/images/8e01cbc6-21bc-11ea-a13a-137349068a90.jpg',
  'max_detection_conf': 0,
  'detections': []},
 {'file': '/kaggle/working/images/anno_~kaggle~working~images~94cf8632-21bc-11ea-a13a-137349068a90.jpg',
  'max_detection_conf': 0.997,
  'detections': [{'category': '1',
    'conf': 0.997,
    'bbox': [0.223, 0.156

In [ ]:
data['detection_categories']

{'1': 'animal', '2': 'person', '3': 'vehicle'}

In [ ]:
data['info']

{'detection_completion_time': '2021-02-23 14:55:09', 'format_version': '1.0'}

Aufgrund der langen Bearbeitungszeit des Object Detectors haben wir uns dafür entschieden, für die weitere Bearbeitung des Projektes mit den fertigen Dateien der Kaggle Challenge zu arbeiten.

Quellen:
* https://github.com/microsoft/CameraTraps/tree/master/detection
* https://github.com/microsoft/CameraTraps/tree/master/visualization
* https://paperswithcode.com/method/inception-resnet-v2